Importing libraries

In [109]:
import numpy as np
import pandas as pd
import seaborn as sns
import sys
import os
import matplotlib.pyplot as plt
import klib

import openaq
from IPython.core.display import display

plt.style.use('dark_background')
plt.rcParams.update({"grid.linewidth":0.5, "grid.alpha":0.5})
sns.set(style='ticks', context='talk')

#Load data

In [110]:
# pd.set_option('max_columns', None)
# pd.set_option('max_rows', None)
# pd.reset_optio('max_rows')
#np.set_printoptions(threshold=sys.maxsize)


daily_aqi_by_county_2017 = pd.read_csv('../../data/raw/daily_aqi_by_county_2017.csv')
daily_aqi_by_county_2018 = pd.read_csv('../../data/raw/daily_aqi_by_county_2018.csv')
daily_aqi_by_county_2019 = pd.read_csv('../../data/raw/daily_aqi_by_county_2019.csv')

annual_income_by_county = pd.read_csv('../../data/raw/CAINC1__ALL_AREAS_1969_2019.csv', encoding='latin-1')

county_demographic = pd.read_csv('../../data/raw/county_cc-est2019-alldata.csv', encoding='latin-1')



# Clean annual personal income by county

Remove columns we do not need and rows not needed

In [111]:
def remove_col(df, col_name):
    '''Returns a dataframe with removed column or columns from old dataframe'''
    new_df = df.copy()

    if(type(col_name) != str and len(col_name) > 1):
        for index in col_name:
            new_df = new_df.drop(str(index), axis=1)
    else:
        new_df = new_df.drop(str(col_name), axis=1)
    return new_df

# used to remove years we do not need
all_incomes_2017_2019 = remove_col(annual_income_by_county, range(1969, 2017))

# used to remove last 4 rows as they are not index data; looks like extra info
all_incomes_2017_2019 = all_incomes_2017_2019[:len(all_incomes_2017_2019) - 4]

# all_incomes_2017_2019.info()
all_incomes_2017_2019

,GeoFIPS,GeoName,Region,TableName,LineCode,IndustryClassification,Description,Unit,2017,2018,2019
0,"""00000""",United States,,CAINC1,1.0,...,Personal income (thousands of dollars),Thousands of dollars,16937582000,17839255000,18542262000
1,"""00000""",United States,,CAINC1,2.0,...,Population (persons) 1/,Number of persons,324985539,326687501,328239523
2,"""00000""",United States,,CAINC1,3.0,...,Per capita personal income (dollars) 2/,Dollars,52118,54606,56490
3,"""01000""",Alabama,5,CAINC1,1.0,...,Personal income (thousands of dollars),Thousands of dollars,199999756,208752683,216449038
4,"""01000""",Alabama,5,CAINC1,2.0,...,Population (persons) 1/,Number of persons,4874486,4887681,4903185
...,...,...,...,...,...,...,...,...,...,...,...
9589,"""97000""",Rocky Mountain,7,CAINC1,2.0,...,Population (persons) 1/,Number of persons,12062055,12233639,12399296
9590,"""97000""",Rocky Mountain,7,CAINC1,3.0,...,Per capita personal income (dollars) 2/,Dollars,49991,52936,54873
9591,"""98000""",Far West,8,CAINC1,1.0,...,Personal income (thousands of dollars),Thousands of dollars,3279057722,3472097346,3634009401
9592,"""98000""",Far West,8,CAINC1,2.0,...,Population (persons) 1/,Number of persons,56059482,56350416,56572426


change column types

In [112]:
# all_incomes_2017_2019['2017'].fillna(np.NaN, inplace=True)
# all_incomes_2017_2019['2017 personal income'] = all_incomes_2017_2019['2017'].astype(float)

# for i, item in enumerate(all_incomes_2017_2019['2017']):
#    try:
#       int(item)
#    except ValueError:
#        print('ERROR at index {}: {!r}'.format(i, item))
#        break

all_incomes_2017_2019['Year 2017 (thousands of dollars)'] = pd.to_numeric(all_incomes_2017_2019['2017'], downcast='float', errors='coerce')
all_incomes_2017_2019['Year 2018 (thousands of dollars)'] = pd.to_numeric(all_incomes_2017_2019['2018'], downcast='float', errors='coerce')
all_incomes_2017_2019['Year 2019 (thousands of dollars)'] = pd.to_numeric(all_incomes_2017_2019['2019'], downcast='float', errors='coerce')
all_incomes_2017_2019.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9594 entries, 0 to 9593
Data columns (total 14 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   GeoFIPS                           9594 non-null   object 
 1   GeoName                           9594 non-null   object 
 2   Region                            9594 non-null   object 
 3   TableName                         9594 non-null   object 
 4   LineCode                          9594 non-null   float64
 5   IndustryClassification            9594 non-null   object 
 6   Description                       9594 non-null   object 
 7   Unit                              9594 non-null   object 
 8   2017                              9594 non-null   object 
 9   2018                              9594 non-null   object 
 10  2019                              9594 non-null   object 
 11  Year 2017 (thousands of dollars)  9519 non-null   float32
 12  Year 2

drop old object columns for years

In [113]:
all_incomes_2017_2019 = remove_col(all_incomes_2017_2019, range(2017,2020))
all_incomes_2017_2019.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9594 entries, 0 to 9593
Data columns (total 11 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   GeoFIPS                           9594 non-null   object 
 1   GeoName                           9594 non-null   object 
 2   Region                            9594 non-null   object 
 3   TableName                         9594 non-null   object 
 4   LineCode                          9594 non-null   float64
 5   IndustryClassification            9594 non-null   object 
 6   Description                       9594 non-null   object 
 7   Unit                              9594 non-null   object 
 8   Year 2017 (thousands of dollars)  9519 non-null   float32
 9   Year 2018 (thousands of dollars)  9519 non-null   float32
 10  Year 2019 (thousands of dollars)  9519 non-null   float32
dtypes: float32(3), float64(1), object(7)
memory usage: 712.2+ KB


Need to deal with the 3 descriptions personal income, population, per capita personal income (dollars)

In [114]:
personal_income = all_incomes_2017_2019['Description'] == ('Personal income (thousands of dollars)')
county_personal_incomes_2017_2019 = all_incomes_2017_2019[personal_income]
county_personal_incomes_2017_2019.head()

,GeoFIPS,GeoName,Region,TableName,LineCode,IndustryClassification,Description,Unit,Year 2017 (thousands of dollars),Year 2018 (thousands of dollars),Year 2019 (thousands of dollars)
0,"""00000""",United States,,CAINC1,1.0,...,Personal income (thousands of dollars),Thousands of dollars,1.693758e+10,1.783926e+10,1.854226e+10
3,"""01000""",Alabama,5,CAINC1,1.0,...,Personal income (thousands of dollars),Thousands of dollars,1.999998e+08,2.087527e+08,2.164490e+08
6,"""01001""","Autauga, AL",5,CAINC1,1.0,...,Personal income (thousands of dollars),Thousands of dollars,2.276561e+06,2.360366e+06,2.453617e+06
9,"""01003""","Baldwin, AL",5,CAINC1,1.0,...,Personal income (thousands of dollars),Thousands of dollars,9.471242e+06,1.006597e+07,1.060026e+07
12,"""01005""","Barbour, AL",5,CAINC1,1.0,...,Personal income (thousands of dollars),Thousands of dollars,8.381840e+05,8.721890e+05,8.828340e+05


* remove more columns
* remove IndustryClassifaction as it only contains ... values and also not what we are interested in
* remove TableName as only one value and not what we are interested in
* remove Unit as that has the same value as Thousands of dollars
* remove Description as that is all personal income now
* remove LineCode as there are no unique values all value is 1

In [115]:
cols_to_remove = ['IndustryClassification', 'TableName', 'Unit', 'Description', 'LineCode']
# county_personal_incomes_2017_2019 = remove_col(personal_income, cols_to_remove)
county_personal_incomes_2017_2019
county_personal_incomes_2017_2019 = remove_col(county_personal_incomes_2017_2019, cols_to_remove)
county_personal_incomes_2017_2019

,GeoFIPS,GeoName,Region,Year 2017 (thousands of dollars),Year 2018 (thousands of dollars),Year 2019 (thousands of dollars)
0,"""00000""",United States,,1.693758e+10,1.783926e+10,1.854226e+10
3,"""01000""",Alabama,5,1.999998e+08,2.087527e+08,2.164490e+08
6,"""01001""","Autauga, AL",5,2.276561e+06,2.360366e+06,2.453617e+06
9,"""01003""","Baldwin, AL",5,9.471242e+06,1.006597e+07,1.060026e+07
12,"""01005""","Barbour, AL",5,8.381840e+05,8.721890e+05,8.828340e+05
...,...,...,...,...,...,...
9579,"""94000""",Plains,4,1.055029e+09,1.107630e+09,1.146515e+09
9582,"""95000""",Southeast,5,3.818755e+09,4.022276e+09,4.173677e+09
9585,"""96000""",Southwest,6,1.924648e+09,2.051027e+09,2.144764e+09
9588,"""97000""",Rocky Mountain,7,6.029942e+08,6.476007e+08,6.803901e+08


In [116]:
county_personal_incomes_2017_2019.info()
type(county_personal_incomes_2017_2019['GeoName'].values[1])

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3198 entries, 0 to 9591
Data columns (total 6 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   GeoFIPS                           3198 non-null   object 
 1   GeoName                           3198 non-null   object 
 2   Region                            3198 non-null   object 
 3   Year 2017 (thousands of dollars)  3173 non-null   float32
 4   Year 2018 (thousands of dollars)  3173 non-null   float32
 5   Year 2019 (thousands of dollars)  3173 non-null   float32
dtypes: float32(3), object(3)
memory usage: 137.4+ KB


str

# Clean AQI pollution

Combine the aqi pollution years together

In [117]:
county_daily_aqi_by_2017_2019 = pd.concat([daily_aqi_by_county_2017, daily_aqi_by_county_2018, daily_aqi_by_county_2019], axis=0)

county_daily_aqi_by_2017_2019.rename(columns={'county Name':'County Name'}, inplace=True)
county_daily_aqi_by_2017_2019


,State Name,County Name,State Code,County Code,Date,AQI,Category,Defining Parameter,Defining Site,Number of Sites Reporting
0,Alabama,Baldwin,1,3,2017-01-01,21,Good,PM2.5,01-003-0010,1
1,Alabama,Baldwin,1,3,2017-01-04,22,Good,PM2.5,01-003-0010,1
2,Alabama,Baldwin,1,3,2017-01-10,19,Good,PM2.5,01-003-0010,1
3,Alabama,Baldwin,1,3,2017-01-13,30,Good,PM2.5,01-003-0010,1
4,Alabama,Baldwin,1,3,2017-01-16,16,Good,PM2.5,01-003-0010,1
...,...,...,...,...,...,...,...,...,...,...
341940,Wyoming,Weston,56,45,2019-12-27,36,Good,Ozone,56-045-0003,2
341941,Wyoming,Weston,56,45,2019-12-28,37,Good,Ozone,56-045-0003,2
341942,Wyoming,Weston,56,45,2019-12-29,34,Good,Ozone,56-045-0003,2
341943,Wyoming,Weston,56,45,2019-12-30,36,Good,Ozone,56-045-0003,2


Change date to datetime object

In [118]:
# county_daily_aqi_by_2017_2019.info()
type(county_daily_aqi_by_2017_2019.Date.values[1])
county_daily_aqi_by_2017_2019.Date = pd.to_datetime(county_daily_aqi_by_2017_2019.Date)
type(county_daily_aqi_by_2017_2019.Date.values[1])

numpy.datetime64

# Clean demographic data
* (clean up on years interested and age group interested in)
* (check if column types are appropriate)
* (remove sumlev col as it just indicates that we are using county level or equivalent)

In [119]:
year_2017_2019 = county_demographic['YEAR'] > 9
county_demographic_2017_2019 = county_demographic.loc[year_2017_2019]
county_demographic_2017_2019 = county_demographic_2017_2019.reset_index(drop=True)
county_demographic_2017_2019 = remove_col(county_demographic_2017_2019, 'SUMLEV')
county_demographic_2017_2019.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 179094 entries, 0 to 179093
Data columns (total 79 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   STATE         179094 non-null  int64 
 1   COUNTY        179094 non-null  int64 
 2   STNAME        179094 non-null  object
 3   CTYNAME       179094 non-null  object
 4   YEAR          179094 non-null  int64 
 5   AGEGRP        179094 non-null  int64 
 6   TOT_POP       179094 non-null  int64 
 7   TOT_MALE      179094 non-null  int64 
 8   TOT_FEMALE    179094 non-null  int64 
 9   WA_MALE       179094 non-null  int64 
 10  WA_FEMALE     179094 non-null  int64 
 11  BA_MALE       179094 non-null  int64 
 12  BA_FEMALE     179094 non-null  int64 
 13  IA_MALE       179094 non-null  int64 
 14  IA_FEMALE     179094 non-null  int64 
 15  AA_MALE       179094 non-null  int64 
 16  AA_FEMALE     179094 non-null  int64 
 17  NA_MALE       179094 non-null  int64 
 18  NA_FEMALE     179094 non

In [120]:
# reduce rows by having a reducing age group to totals
# maybe try binning age groups from 1-12 (0-59) and 13-19 (60+)
total = county_demographic_2017_2019.AGEGRP == 0
county_demographic_2017_2019 = county_demographic_2017_2019.loc[total]
county_demographic_2017_2019 = remove_col(county_demographic_2017_2019, 'AGEGRP')


Need to reduce ethnicity columns by combining similar or dropping irrelevant columns

In [121]:
ethnicities = list(county_demographic_2017_2019.iloc[:, 20:29])
ethnicities.extend(list(county_demographic_2017_2019.iloc[:, 32:54]))
ethnicities.extend(list(county_demographic_2017_2019.iloc[:, 56:]))
county_demographic_2017_2019 = remove_col(county_demographic_2017_2019, ethnicities)

# can add columns

In [122]:
display(county_demographic_2017_2019)


,STATE,COUNTY,STNAME,CTYNAME,YEAR,TOT_POP,TOT_MALE,TOT_FEMALE,WA_MALE,WA_FEMALE,...,AA_FEMALE,NA_MALE,NA_FEMALE,TOM_MALE,TOM_FEMALE,NAC_FEMALE,NH_MALE,NH_FEMALE,H_MALE,H_FEMALE
0,1,1,Alabama,Autauga County,10,55390,27010,28380,20876,21705,...,385,30,26,561,481,55,26188,27643,822,737
19,1,1,Alabama,Autauga County,11,55533,27006,28527,20917,21749,...,374,33,28,535,494,58,26140,27771,866,756
38,1,1,Alabama,Autauga County,12,55869,27092,28777,20878,21729,...,370,32,26,538,507,49,26208,27990,884,787
57,1,3,Alabama,Baldwin County,10,212521,103218,109303,90360,95215,...,1361,69,69,1785,1913,168,98087,104753,5131,4550
76,1,3,Alabama,Baldwin County,11,217855,105702,112153,92613,97815,...,1396,70,72,1897,1984,176,100355,107363,5347,4790
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
178999,56,43,Wyoming,Washakie County,11,7877,3984,3893,3781,3670,...,37,5,3,86,90,8,3416,3352,568,541
179018,56,43,Wyoming,Washakie County,12,7805,3963,3842,3759,3618,...,39,4,2,87,89,11,3398,3299,565,543
179037,56,45,Wyoming,Weston County,10,6968,3660,3308,3447,3111,...,63,2,0,78,75,7,3501,3180,159,128
179056,56,45,Wyoming,Weston County,11,6924,3627,3297,3401,3073,...,74,2,0,83,84,3,3475,3176,152,121


# Combine the three separate dataframes
Attempt to combine dataframe through creating foreign key, GeoFIPs

In [123]:
display(county_demographic_2017_2019)
display(county_personal_incomes_2017_2019)
display(county_daily_aqi_by_2017_2019)
# print(county_demographic_2017_2019.loc[county_demographic_2017_2019.COUNTY == 3])
# make a foreign key to link them all up

,STATE,COUNTY,STNAME,CTYNAME,YEAR,TOT_POP,TOT_MALE,TOT_FEMALE,WA_MALE,WA_FEMALE,...,AA_FEMALE,NA_MALE,NA_FEMALE,TOM_MALE,TOM_FEMALE,NAC_FEMALE,NH_MALE,NH_FEMALE,H_MALE,H_FEMALE
0,1,1,Alabama,Autauga County,10,55390,27010,28380,20876,21705,...,385,30,26,561,481,55,26188,27643,822,737
19,1,1,Alabama,Autauga County,11,55533,27006,28527,20917,21749,...,374,33,28,535,494,58,26140,27771,866,756
38,1,1,Alabama,Autauga County,12,55869,27092,28777,20878,21729,...,370,32,26,538,507,49,26208,27990,884,787
57,1,3,Alabama,Baldwin County,10,212521,103218,109303,90360,95215,...,1361,69,69,1785,1913,168,98087,104753,5131,4550
76,1,3,Alabama,Baldwin County,11,217855,105702,112153,92613,97815,...,1396,70,72,1897,1984,176,100355,107363,5347,4790
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
178999,56,43,Wyoming,Washakie County,11,7877,3984,3893,3781,3670,...,37,5,3,86,90,8,3416,3352,568,541
179018,56,43,Wyoming,Washakie County,12,7805,3963,3842,3759,3618,...,39,4,2,87,89,11,3398,3299,565,543
179037,56,45,Wyoming,Weston County,10,6968,3660,3308,3447,3111,...,63,2,0,78,75,7,3501,3180,159,128
179056,56,45,Wyoming,Weston County,11,6924,3627,3297,3401,3073,...,74,2,0,83,84,3,3475,3176,152,121


,GeoFIPS,GeoName,Region,Year 2017 (thousands of dollars),Year 2018 (thousands of dollars),Year 2019 (thousands of dollars)
0,"""00000""",United States,,1.693758e+10,1.783926e+10,1.854226e+10
3,"""01000""",Alabama,5,1.999998e+08,2.087527e+08,2.164490e+08
6,"""01001""","Autauga, AL",5,2.276561e+06,2.360366e+06,2.453617e+06
9,"""01003""","Baldwin, AL",5,9.471242e+06,1.006597e+07,1.060026e+07
12,"""01005""","Barbour, AL",5,8.381840e+05,8.721890e+05,8.828340e+05
...,...,...,...,...,...,...
9579,"""94000""",Plains,4,1.055029e+09,1.107630e+09,1.146515e+09
9582,"""95000""",Southeast,5,3.818755e+09,4.022276e+09,4.173677e+09
9585,"""96000""",Southwest,6,1.924648e+09,2.051027e+09,2.144764e+09
9588,"""97000""",Rocky Mountain,7,6.029942e+08,6.476007e+08,6.803901e+08


,State Name,County Name,State Code,County Code,Date,AQI,Category,Defining Parameter,Defining Site,Number of Sites Reporting
0,Alabama,Baldwin,1,3,2017-01-01,21,Good,PM2.5,01-003-0010,1
1,Alabama,Baldwin,1,3,2017-01-04,22,Good,PM2.5,01-003-0010,1
2,Alabama,Baldwin,1,3,2017-01-10,19,Good,PM2.5,01-003-0010,1
3,Alabama,Baldwin,1,3,2017-01-13,30,Good,PM2.5,01-003-0010,1
4,Alabama,Baldwin,1,3,2017-01-16,16,Good,PM2.5,01-003-0010,1
...,...,...,...,...,...,...,...,...,...,...
341940,Wyoming,Weston,56,45,2019-12-27,36,Good,Ozone,56-045-0003,2
341941,Wyoming,Weston,56,45,2019-12-28,37,Good,Ozone,56-045-0003,2
341942,Wyoming,Weston,56,45,2019-12-29,34,Good,Ozone,56-045-0003,2
341943,Wyoming,Weston,56,45,2019-12-30,36,Good,Ozone,56-045-0003,2


In [124]:
# need to make a foreign key for ethnicity and aqi
state_dem = county_demographic_2017_2019.STATE.astype(str).values
county_dem = county_demographic_2017_2019.COUNTY.astype(str).values
geofips_dem = []
for stateID, countyID in zip(state_dem, county_dem):
    id = ''
    stateLen = len(stateID)
    countyLen = len(countyID)
    if(stateLen < 2):
        id += '0' + stateID
    else:
        id += stateID
    if(countyLen == 1):
        id += '00' + countyID
    elif(countyLen == 2):
        id += '0' + countyID
    else:
        id += countyID
    geofips_dem.append(id)

county_demographic_2017_2019['GeoFIPS'] = geofips_dem
county_demographic_2017_2019

,STATE,COUNTY,STNAME,CTYNAME,YEAR,TOT_POP,TOT_MALE,TOT_FEMALE,WA_MALE,WA_FEMALE,...,NA_MALE,NA_FEMALE,TOM_MALE,TOM_FEMALE,NAC_FEMALE,NH_MALE,NH_FEMALE,H_MALE,H_FEMALE,GeoFIPS
0,1,1,Alabama,Autauga County,10,55390,27010,28380,20876,21705,...,30,26,561,481,55,26188,27643,822,737,01001
19,1,1,Alabama,Autauga County,11,55533,27006,28527,20917,21749,...,33,28,535,494,58,26140,27771,866,756,01001
38,1,1,Alabama,Autauga County,12,55869,27092,28777,20878,21729,...,32,26,538,507,49,26208,27990,884,787,01001
57,1,3,Alabama,Baldwin County,10,212521,103218,109303,90360,95215,...,69,69,1785,1913,168,98087,104753,5131,4550,01003
76,1,3,Alabama,Baldwin County,11,217855,105702,112153,92613,97815,...,70,72,1897,1984,176,100355,107363,5347,4790,01003
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
178999,56,43,Wyoming,Washakie County,11,7877,3984,3893,3781,3670,...,5,3,86,90,8,3416,3352,568,541,56043
179018,56,43,Wyoming,Washakie County,12,7805,3963,3842,3759,3618,...,4,2,87,89,11,3398,3299,565,543,56043
179037,56,45,Wyoming,Weston County,10,6968,3660,3308,3447,3111,...,2,0,78,75,7,3501,3180,159,128,56045
179056,56,45,Wyoming,Weston County,11,6924,3627,3297,3401,3073,...,2,0,83,84,3,3475,3176,152,121,56045


In [125]:
#repeat for aqi
state_aqi = county_daily_aqi_by_2017_2019['State Code'].astype(str).values
county_aqi = county_daily_aqi_by_2017_2019['County Code'].astype(str).values

geofips_aqi = []
for stateID, countyID in zip(state_aqi, county_aqi):
    id = ''
    stateLen = len(stateID)
    countyLen = len(countyID)
    if(stateLen < 2):
        id += '0' + stateID
    else:
        id += stateID
    if(countyLen == 1):
        id += '00' + countyID
    elif(countyLen == 2):
        id += '0' + countyID
    else:
        id += countyID
    geofips_aqi.append(id)

county_daily_aqi_by_2017_2019['GeoFIPS'] = geofips_aqi
county_daily_aqi_by_2017_2019

# pd.set_option('display.max_rows', 1000)
# county_personal_incomes_2017_2019

,State Name,County Name,State Code,County Code,Date,AQI,Category,Defining Parameter,Defining Site,Number of Sites Reporting,GeoFIPS
0,Alabama,Baldwin,1,3,2017-01-01,21,Good,PM2.5,01-003-0010,1,01003
1,Alabama,Baldwin,1,3,2017-01-04,22,Good,PM2.5,01-003-0010,1,01003
2,Alabama,Baldwin,1,3,2017-01-10,19,Good,PM2.5,01-003-0010,1,01003
3,Alabama,Baldwin,1,3,2017-01-13,30,Good,PM2.5,01-003-0010,1,01003
4,Alabama,Baldwin,1,3,2017-01-16,16,Good,PM2.5,01-003-0010,1,01003
...,...,...,...,...,...,...,...,...,...,...,...
341940,Wyoming,Weston,56,45,2019-12-27,36,Good,Ozone,56-045-0003,2,56045
341941,Wyoming,Weston,56,45,2019-12-28,37,Good,Ozone,56-045-0003,2,56045
341942,Wyoming,Weston,56,45,2019-12-29,34,Good,Ozone,56-045-0003,2,56045
341943,Wyoming,Weston,56,45,2019-12-30,36,Good,Ozone,56-045-0003,2,56045


In [126]:
# remove "" characters from income dataset
county_personal_incomes_2017_2019['temp'] = county_personal_incomes_2017_2019.GeoFIPS.str.strip('" "')
county_personal_incomes_2017_2019.GeoFIPS = county_personal_incomes_2017_2019.temp
county_personal_incomes_2017_2019 = remove_col(county_personal_incomes_2017_2019, 'temp')
county_personal_incomes_2017_2019

,GeoFIPS,GeoName,Region,Year 2017 (thousands of dollars),Year 2018 (thousands of dollars),Year 2019 (thousands of dollars)
0,00000,United States,,1.693758e+10,1.783926e+10,1.854226e+10
3,01000,Alabama,5,1.999998e+08,2.087527e+08,2.164490e+08
6,01001,"Autauga, AL",5,2.276561e+06,2.360366e+06,2.453617e+06
9,01003,"Baldwin, AL",5,9.471242e+06,1.006597e+07,1.060026e+07
12,01005,"Barbour, AL",5,8.381840e+05,8.721890e+05,8.828340e+05
...,...,...,...,...,...,...
9579,94000,Plains,4,1.055029e+09,1.107630e+09,1.146515e+09
9582,95000,Southeast,5,3.818755e+09,4.022276e+09,4.173677e+09
9585,96000,Southwest,6,1.924648e+09,2.051027e+09,2.144764e+09
9588,97000,Rocky Mountain,7,6.029942e+08,6.476007e+08,6.803901e+08


# Backtrack: Check for missing data
Check for any missing data or other areas before merginng the datasets

In [127]:
display(county_demographic_2017_2019.isnull().sum())
display(county_daily_aqi_by_2017_2019.isnull().sum())
display(county_personal_incomes_2017_2019.isnull().sum())

STATE         0
COUNTY        0
STNAME        0
CTYNAME       0
YEAR          0
TOT_POP       0
TOT_MALE      0
TOT_FEMALE    0
WA_MALE       0
WA_FEMALE     0
BA_MALE       0
BA_FEMALE     0
IA_MALE       0
IA_FEMALE     0
AA_MALE       0
AA_FEMALE     0
NA_MALE       0
NA_FEMALE     0
TOM_MALE      0
TOM_FEMALE    0
NAC_FEMALE    0
NH_MALE       0
NH_FEMALE     0
H_MALE        0
H_FEMALE      0
GeoFIPS       0
dtype: int64

State Name                   0
County Name                  0
State Code                   0
County Code                  0
Date                         0
AQI                          0
Category                     0
Defining Parameter           0
Defining Site                0
Number of Sites Reporting    0
GeoFIPS                      0
dtype: int64

GeoFIPS                              0
GeoName                              0
Region                               0
Year 2017 (thousands of dollars)    25
Year 2018 (thousands of dollars)    25
Year 2019 (thousands of dollars)    25
dtype: int64

In [128]:
# missing data in personal income

missing = county_personal_incomes_2017_2019['Year 2017 (thousands of dollars)'].isnull()
display(county_personal_incomes_2017_2019.loc[missing])
missing_id = county_personal_incomes_2017_2019.loc[missing]['GeoFIPS'].values
# missing data mainly in AK and one in Wisconsin
# doesnt really to matter much as they are not found in other dataset and cannot be merge

,GeoFIPS,GeoName,Region,Year 2017 (thousands of dollars),Year 2018 (thousands of dollars),Year 2019 (thousands of dollars)
210,02010,"Aleutian Islands Census Area, AK*",8,NaN,NaN,NaN
279,02201,"Prince of Wales-Outer Ketchikan Census Area, AK*",8,NaN,NaN,NaN
288,02231,"Skagway-Yakutat-Angoon Census Area, AK*",8,NaN,NaN,NaN
291,02232,"Skagway-Hoonah-Angoon Census Area, AK*",8,NaN,NaN,NaN
303,02280,"Wrangell-Petersburg Census Area, AK*",8,NaN,NaN,NaN
312,02901,"Aleutian Islands Division, AK*",8,NaN,NaN,NaN
315,02903,"Angoon Division, AK*",8,NaN,NaN,NaN
318,02904,"Barrow-North Slope Division, AK*",8,NaN,NaN,NaN
321,02905,"Bethel Division, AK*",8,NaN,NaN,NaN
324,02907,"Bristol Bay Division, AK*",8,NaN,NaN,NaN


In [129]:
#inspect missing data from other datasets

for id in missing_id:
    display(id in county_daily_aqi_by_2017_2019['GeoFIPS'])
    display(id in county_demographic_2017_2019['GeoFIPS'])

False

False

False

False

False

...

False

False

False

False

False

Missing data for these counties are not found in the other datasets; and will be removed when merged
Means our dataset will not have any missing data


In [130]:
# merge dataframes by geofips
# merge income and ethnicity
county_income_ethnicity_2017_2019 = pd.merge(county_personal_incomes_2017_2019, county_demographic_2017_2019, on='GeoFIPS')


In [131]:
df = pd.merge(county_income_ethnicity_2017_2019, county_daily_aqi_by_2017_2019, on='GeoFIPS')

In [132]:
# need to remove dupe columns
cols_to_remove = ['GeoName', 'STATE', 'COUNTY', 'STNAME', 'CTYNAME', 'State Code', 'County Code']
df = remove_col(df, cols_to_remove)

In [133]:
# print(df.columns)
# df.head(50)
df = df[['GeoFIPS', 'Region', 'State Name', 'County Name', 'Year 2017 (thousands of dollars)',
       'Year 2018 (thousands of dollars)', 'Year 2019 (thousands of dollars)',
       'YEAR', 'TOT_POP', 'TOT_MALE', 'TOT_FEMALE', 'WA_MALE', 'WA_FEMALE',
       'BA_MALE', 'BA_FEMALE', 'IA_MALE', 'IA_FEMALE', 'AA_MALE', 'AA_FEMALE',
       'NA_MALE', 'NA_FEMALE', 'TOM_MALE', 'TOM_FEMALE', 'NAC_FEMALE',
       'NH_MALE', 'NH_FEMALE', 'H_MALE', 'H_FEMALE', 'Date', 'AQI', 'Category', 'Defining Parameter',
       'Defining Site', 'Number of Sites Reporting']]

In [134]:
# df.isnull().sum()
# AK = df['State Name'] == 'Arkansas'
# df.head()

save data for more cleaning


In [137]:
# save the data to a new csv file. keep commented to prevent from always saving

# data_path = r'../../data/processed/clean_county_2017_2019.csv'
# df.to_csv(data_path)
